In [1]:
# run dependencies

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import create_engine

In [2]:
# Import dataframes that give features

In [ ]:
#call all csv files that were created (feature files)
wins = pd.read_csv("Features/total_wins.csv")
appearances = pd.read_csv("Features/adv_df_pivot.csv")
rank = pd.read_csv("Features/fifa_rank.csv")
delta_pts = pd.read_csv("Features/delta_pts.csv")
